**2023-2024 流動電費計價函數與使用方法**

**version:**
- 111H2: 111年7月1日起實施
- 112H1: 112年4月1日起實施
- 112H2: 112年11月1日起實施
- 113H1: 113年4月1日起實施
- 113H2: 113年10月16日起實施

**summer_rule:**
- 低壓/EV: 6/1–9/30
- 高壓: 111H2 為 6/1–9/30，≥112H1 起為 5/16–10/15

**kind:**
- 建築電價
    - building_low_voltage: 低壓電力電價
    - building_high_voltage_2stage: 高壓二段式時間電價 
    - building_high_voltage_3stage_fixed_peak: 高壓三段式時間電價(尖峰時間固定)
- 電動車充換電設施電價
    - ev_low_contract: 契約容量未滿100瓩
    - ev_high_contract: 專供電動車充換電及附屬設備之用電，採高壓供電、契約容量在100瓩以上
    - ev_high_contract_hv95: 以高壓供電者，按本表單價95%計算 = ev_high_contract * 95%



In [3]:
import json
from datetime import datetime, time

# --------------------------------------------------------------
# 工具：解析時間段 "HH:MM-HH:MM" → list of hours
# --------------------------------------------------------------
def parse_time_ranges(range_str):
    hours = []
    segments = range_str.split(",")
    for seg in segments:
        start_str, end_str = seg.split("-")
        start_h, start_m = map(int, start_str.split(":"))
        end_h, end_m = map(int, end_str.split(":"))

        start_index = start_h
        end_index = end_h if end_m == 0 else end_h
        if end_h == 24:
            end_index = 23

        for h in range(start_index, end_index + 1):
            hours.append(h)

    return sorted(set(hours))


# --------------------------------------------------------------
# 工具：weekday → weekday / saturday / holiday
# --------------------------------------------------------------
def determine_day_type(date_obj):
    wd = date_obj.weekday()
    if wd == 6:
        return "holiday"
    elif wd == 5:
        return "saturday"
    else:
        return "weekday"


# --------------------------------------------------------------
# 主函數：產生當天 24 小時電價
# --------------------------------------------------------------
def get_daily_tariff(date_str, kind, ALL_TARIFFS):
    date_obj = datetime.strptime(date_str, "%Y-%m-%d").date()

    # ----------------------------------------------------------
    # 支援 kind = "ev_high_contract_hv95"
    # ----------------------------------------------------------
    apply_hv95 = False
    if kind.endswith("_hv95"):
        apply_hv95 = True
        base_kind = kind.replace("_hv95", "")
        kind = base_kind  # 用原始 kind 查表

    # ----------------------------------------------------------
    # 1. 找對 period 的 JSON
    # ----------------------------------------------------------
    matched_json = None
    for key, data in ALL_TARIFFS.items():
        p = data["period"]
        start = datetime.strptime(p["start"], "%Y-%m-%d").date()
        end = datetime.strptime(p["end"], "%Y-%m-%d").date()

        if start <= date_obj <= end:
            matched_json = data
            break

    if matched_json is None:
        raise ValueError("找不到此日期適用的電價版本")

    tariffs = matched_json["tariffs"]

    if kind not in tariffs:
        raise ValueError(f"種類 {kind} 不存在於此版本電價中")

    tariff_rule = tariffs[kind]

    # ----------------------------------------------------------
    # 2. 判斷夏季/非夏季
    # ----------------------------------------------------------
    month = date_obj.month
    day = date_obj.day

    if (month > 6 and month < 9) or (month == 6 and day >= 1) or (month == 9 and day <= 30):
        season = "summer"
    else:
        season = "nonsummer"

    # ----------------------------------------------------------
    # 3. weekday / saturday / holiday
    # ----------------------------------------------------------
    day_type = determine_day_type(date_obj)

    # ----------------------------------------------------------
    # 4. 產生 24 小時電價
    # ----------------------------------------------------------
    prices = [None] * 24
    energy_rule = tariff_rule["energy_charge"][season]

    # 無時段（低壓型）
    if isinstance(energy_rule, (int, float)):
        prices = [energy_rule] * 24
    else:
        # 有時段：依 day_type 選擇
        if day_type not in energy_rule:
            if "holiday" in energy_rule:
                day_type = "holiday"
            else:
                raise ValueError(f"{kind} 在 {day_type} 沒有 energy_charge 設定")

        day_rule = energy_rule[day_type]

        # 依 peak / offpeak / half_peak 填進 prices
        for period_type, (time_range, price) in day_rule.items():
            hours = parse_time_ranges(time_range)
            for h in hours:
                prices[h] = price

        # fill missing
        for i in range(24):
            if prices[i] is None:
                prices[i] = 0.0

    # ----------------------------------------------------------
    # 5. 若是 hv95 → overall price *0.95
    # ----------------------------------------------------------
    if apply_hv95:
        prices = [p * 0.95 for p in prices]

    return prices


**Input:**
- date_str：2023-2024 年度的某一天
- kind：電價種類
    - building_low_voltage
    - building_high_voltage_2stage 
    - building_high_voltage_3stage_fixed_peak
    - ev_low_contract 
    - ev_high_contract 
    - ev_high_contract_hv95

**Output:**
- 產生當天 24 小時電價


In [4]:
ALL_TARIFFS = {
    "111H2": json.load(open("111H2.json", encoding="utf8")),
    "112H1": json.load(open("112H1.json", encoding="utf8")),
    "112H2": json.load(open("112H2.json", encoding="utf8")),
    "113H1": json.load(open("113H1.json", encoding="utf8")),
    "113H2": json.load(open("113H2.json", encoding="utf8"))
}

prices = get_daily_tariff(
    date_str="2023-08-15",
    kind="building_high_voltage_2stage",
    ALL_TARIFFS=ALL_TARIFFS
)

print(prices)


[2.03, 2.03, 2.03, 2.03, 2.03, 2.03, 2.03, 2.03, 2.03, 2.03, 5.05, 5.05, 5.05, 5.05, 5.05, 5.05, 5.05, 5.05, 5.05, 5.05, 5.05, 5.05, 5.05, 5.05]
